<a href="https://colab.research.google.com/github/iskra3138/UDA/blob/main/office31_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sun Mar 21 04:31:12 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Downloading source and dataset

### source files

In [2]:
!git clone https://github.com/tim-learn/SHOT.git

Cloning into 'SHOT'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 227 (delta 11), reused 15 (delta 6), pack-reused 203
Receiving objects: 100% (227/227), 489.24 KiB | 24.46 MiB/s, done.
Resolving deltas: 100% (97/97), done.


In [3]:
cd SHOT/object/

/content/SHOT/object


### office31 dataset

In [4]:
!gdown  https://drive.google.com/uc?id=0B4IapRTv9pJ1WGZVd1VDMmhwdlE

Downloading...
From: https://drive.google.com/uc?id=0B4IapRTv9pJ1WGZVd1VDMmhwdlE
To: /content/SHOT/object/domain_adaptation_images.tar.gz
77.2MB [00:01, 72.0MB/s]


In [5]:
!tar -xvzf domain_adaptation_images.tar.gz

amazon/images/back_pack/frame_0001.jpg
amazon/images/back_pack/frame_0002.jpg
amazon/images/back_pack/frame_0003.jpg
amazon/images/back_pack/frame_0004.jpg
amazon/images/back_pack/frame_0005.jpg
amazon/images/back_pack/frame_0006.jpg
amazon/images/back_pack/frame_0007.jpg
amazon/images/back_pack/frame_0008.jpg
amazon/images/back_pack/frame_0009.jpg
amazon/images/back_pack/frame_0010.jpg
amazon/images/back_pack/frame_0011.jpg
amazon/images/back_pack/frame_0012.jpg
amazon/images/back_pack/frame_0013.jpg
amazon/images/back_pack/frame_0014.jpg
amazon/images/back_pack/frame_0015.jpg
amazon/images/back_pack/frame_0016.jpg
amazon/images/back_pack/frame_0017.jpg
amazon/images/back_pack/frame_0018.jpg
amazon/images/back_pack/frame_0019.jpg
amazon/images/back_pack/frame_0020.jpg
amazon/images/back_pack/frame_0021.jpg
amazon/images/back_pack/frame_0022.jpg
amazon/images/back_pack/frame_0023.jpg
amazon/images/back_pack/frame_0024.jpg
amazon/images/back_pack/frame_0025.jpg
amazon/images/back_pack/f

In [6]:
ll

total 75504
drwxr-xr-x 3 root     4096 Mar 21 04:31 amazon/
-rw-r--r-- 1 root     2797 Mar 21 04:31 data_list.py
-rw-r--r-- 1 root 77187046 Mar 21 04:31 domain_adaptation_images.tar.gz
drwxr-xr-x 3 root     4096 Mar 21 04:31 dslr/
-rw-r--r-- 1 root     9903 Mar 21 04:31 image_multisource.py
-rw-r--r-- 1 root    14024 Mar 21 04:31 image_multitarget.py
-rw-r--r-- 1 root    11369 Mar 21 04:31 image_pretrained.py
-rw-r--r-- 1 root    16232 Mar 21 04:31 image_source.py
-rw-r--r-- 1 root    16393 Mar 21 04:31 image_target_oda.py
-rw-r--r-- 1 root    15741 Mar 21 04:31 image_target.py
-rw-r--r-- 1 root     3361 Mar 21 04:31 loss.py
-rw-r--r-- 1 root     5193 Mar 21 04:31 network.py
-rw-r--r-- 1 root     3312 Mar 21 04:31 run.sh
drwxr-xr-x 3 root     4096 Mar 21 04:31 webcam/


# Data Pipeline

In [7]:
# Making files about paths of images
# ref: https://github.com/tim-learn/SHOT/issues/2 

import os, glob 
data_path = './data/office'
if not os.path.exists(data_path) :
  os.makedirs(data_path)

datasets = ['amazon', 'dslr', 'webcam']
classes = os.listdir('./amazon/images/')

for dataset in datasets:
  img_paths_file = open(os.path.join(data_path,'{}_list.txt'.format(dataset)), 'w')
  for i, cls in enumerate(classes): 
    for filename in glob.glob(os.path.join(dataset,'images',cls,'*.jpg')):
      img_paths_file.write("{} {}\n".format(filename,i))
  img_paths_file.close()

In [8]:
NUM_WORKERS = 2
BATCHSIZE = 64
folder = './data/'
names = ['amazon', 'dslr', 'webcam']
CLASS_NUM = 31
SOURCE = names[1]
TARGET = names[0]
TRTE = 'val'
DSET = 'office'
NET = 'resnet50'
MAX_EPOCH=100

NAME_SRC = SOURCE[0].upper()

S_DSET_PATH = folder + DSET + '/' + '{}_list.txt'.format(SOURCE)
TEST_DSET_PATH = folder + DSET + '/' + '{}_list.txt'.format(TARGET)

In [9]:
import os
import torchvision
import numpy as np
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
from data_list import ImageList
import random

SEED = 0
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)
random.seed(SEED)


def image_train(resize_size=256, crop_size=224, alexnet=False):
  if not alexnet:
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                   std=[0.229, 0.224, 0.225])
  else:
    normalize = Normalize(meanfile='./ilsvrc_2012_mean.npy')
  return  transforms.Compose([
        transforms.Resize((resize_size, resize_size)),
        transforms.RandomCrop(crop_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ])

def image_test(resize_size=256, crop_size=224, alexnet=False):
  if not alexnet:
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                   std=[0.229, 0.224, 0.225])
  else:
    normalize = Normalize(meanfile='./ilsvrc_2012_mean.npy')
  return  transforms.Compose([
        transforms.Resize((resize_size, resize_size)),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor(),
        normalize
    ])
  
def data_load(): 
    ## prepare data
    dsets = {}
    dset_loaders = {}
    train_bs = BATCHSIZE
    txt_src = open(S_DSET_PATH).readlines()
    txt_test = open(TEST_DSET_PATH).readlines()

    if TRTE == "val":
        dsize = len(txt_src)
        tr_size = int(0.9*dsize)
        # print(dsize, tr_size, dsize - tr_size)
        tr_txt, te_txt = torch.utils.data.random_split(txt_src, [tr_size, dsize - tr_size])
    else:
        dsize = len(txt_src)
        tr_size = int(0.9*dsize)
        _, te_txt = torch.utils.data.random_split(txt_src, [tr_size, dsize - tr_size])
        tr_txt = txt_src

    dsets["source_tr"] = ImageList(tr_txt, transform=image_train())
    dset_loaders["source_tr"] = DataLoader(dsets["source_tr"], batch_size=train_bs, shuffle=True, num_workers=NUM_WORKERS, drop_last=False)
    dsets["source_te"] = ImageList(te_txt, transform=image_test())
    dset_loaders["source_te"] = DataLoader(dsets["source_te"], batch_size=train_bs, shuffle=True, num_workers=NUM_WORKERS, drop_last=False)
    dsets["test"] = ImageList(txt_test, transform=image_test())
    dset_loaders["test"] = DataLoader(dsets["test"], batch_size=train_bs*2, shuffle=True, num_workers=NUM_WORKERS, drop_last=False)

    return dset_loaders


In [10]:
dset_loaders = data_load()

acc_init = 0
max_iter = MAX_EPOCH * len(dset_loaders["source_tr"])
interval_iter = max_iter // 10
iter_num = 0

# Training by Tensorflow2

In [11]:
!pip install -q -U tensorflow-addons

     |████████████████████████████████| 706kB 2.7MB/s 


In [12]:
import tensorflow as tf
import tensorflow_addons as tfa
print("Tensorflow version " + tf.__version__)
IMAGE_SIZE = (224,224)
AUTO = tf.data.experimental.AUTOTUNE

Tensorflow version 2.4.1


In [13]:
def create_model():
    pretrained_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=[*IMAGE_SIZE, 3])
    #pretrained_model = tf.keras.applications.ResNet50(include_top=False, input_shape=[*IMAGE_SIZE, 3])
    pretrained_model.trainable = True
    
    x = pretrained_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(256, 
                              kernel_initializer='glorot_normal', 
                              bias_initializer='zeros',
                              name='feature') (x)  
    x = tf.keras.layers.BatchNormalization()(x)
    #x = tf.keras.layers.ReLU()(x)  
    #x = tf.keras.layers.Dropout(rate=0.5)(x)
    predictions = tfa.layers.WeightNormalization(tf.keras.layers.Dense(31, 
                                                                       kernel_initializer='glorot_normal', 
                                                                       bias_initializer='zeros',
                                                                       activation='softmax', 
                                                                       name='prediction')) (x)
    #predictions = tf.keras.layers.Dense(31, name='prediction') (x)
    model = tf.keras.Model(inputs=pretrained_model.input, outputs=predictions)

    return model

In [14]:
model = create_model()
model.summary()

94773248/94765736 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_____________________________

In [15]:
model.losses

[]

In [16]:
# adding L2 regularizer (https://sthalles.github.io/keras-regularizer/)
import os
import tempfile

def add_regularization(model, regularizer=tf.keras.regularizers.l2(0.001)):

    if not isinstance(regularizer, tf.keras.regularizers.Regularizer):
      print("Regularizer must be a subclass of tf.keras.regularizers.Regularizer")
      return model

    for layer in model.layers:
        for attr in ['kernel_regularizer']:
            if hasattr(layer, attr):
              setattr(layer, attr, regularizer)

    # When we change the layers attributes, the change only happens in the model config file
    model_json = model.to_json()

    # Save the weights before reloading the model.
    tmp_weights_path = os.path.join(tempfile.gettempdir(), 'tmp_weights.h5')
    model.save_weights(tmp_weights_path)

    # load the model from the config
    model = tf.keras.models.model_from_json(model_json)
    
    # Reload the model weights
    model.load_weights(tmp_weights_path, by_name=True)
    return model

In [17]:
model = add_regularization(model)

In [18]:
model.losses

[<tf.Tensor: shape=(), dtype=float32, numpy=0.11616695>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.021069562>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0339693>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.05057001>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.025638832>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.019488763>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.038293593>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.022783704>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.020211041>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.036940254>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.021386005>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.029886987>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.07406923>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.093878016>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.05633305>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.036226183>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.06651061>,
 <tf.Tensor: shape=()

In [19]:
import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow.python.ops import math_ops

class CustomDecay(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(
      self,
      initial_learning_rate,
      max_iter,
      gamma=10,
      power=0.75,
      name=None):
    """Applies exponential decay to the learning rate.
    Args:
      initial_learning_rate: A scalar `float32` or `float64` `Tensor` or a
        Python number.  The initial learning rate.
      decay_steps: A scalar `int32` or `int64` `Tensor` or a Python number.
        Must be positive.  See the decay computation above.
      decay_rate: A scalar `float32` or `float64` `Tensor` or a
        Python number.  The decay rate.
      staircase: Boolean.  If `True` decay the learning rate at discrete
        intervals
      name: String.  Optional name of the operation.  Defaults to
        'ExponentialDecay'.
    """
    super(CustomDecay, self).__init__()
    self.initial_learning_rate = initial_learning_rate
    self.max_iter = max_iter
    self.gamma = gamma
    self.power = power
    self.name = name

  def __call__(self, step):
    with ops.name_scope_v2(self.name or "lrDecay") as name:
      initial_learning_rate = ops.convert_to_tensor_v2_with_dispatch(
          self.initial_learning_rate, name="initial_learning_rate")
      dtype = initial_learning_rate.dtype

      max_iter = math_ops.cast(self.max_iter, dtype)
      gamma = math_ops.cast(self.gamma, dtype)
      power = math_ops.cast(self.power, dtype)

      global_step_recomp = math_ops.cast(step, dtype)
      p = global_step_recomp / max_iter
      
      decay = math_ops.pow(math_ops.add(1.0,math_ops.multiply(gamma, p)),math_ops.multiply(-1.0,power))

      return math_ops.multiply(
          initial_learning_rate, decay, name=name)

  def get_config(self):
    return {
        "initial_learning_rate": self.initial_learning_rate,
        "max_iter": self.max_iter,
        "gamma": self.gamma,
        "power": self.power,
        "name": self.name
    }

In [20]:
#loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, )
loss_object = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1)
#optimizer = tf.keras.optimizers.Adam()
#optimizer = tf.keras.optimizers.SGD(momentum=0.9, nesterov=True)

initial_learning_rate = 0.1
lr_schedule1 = CustomDecay(
    initial_learning_rate=0.001,
    max_iter = max_iter,
    gamma=10,
    power=0.75)

lr_schedule2 = CustomDecay(
    initial_learning_rate=0.01,
    max_iter = max_iter,
    gamma=10,
    power=0.75)

optimizer1 = tf.keras.optimizers.SGD(learning_rate=lr_schedule1, momentum=0.9, nesterov=True)
optimizer2 = tf.keras.optimizers.SGD(learning_rate=lr_schedule2, momentum=0.9, nesterov=True)

In [21]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

In [22]:
@tf.function
def train_step(images, labels):
  '''
  variable_list1 =[]
  variable_list2 =[]

  for var in model.trainable_variables:
    if 'conv' in var.name :
      variable_list1.append(var)
    else:
      variable_list2.append(var)
  '''    
  variable_list1 = [var for var in model.trainable_variables if 'conv' in var.name]
  variable_list2 = [var for var in model.trainable_variables if not 'conv' in var.name]

  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    labels = tf.one_hot(labels, 31)
    loss = loss_object(labels, predictions)

  gradients = tape.gradient(loss, variable_list1 + variable_list2)
  grads1 = gradients[:len(variable_list1)]
  grads2 = gradients[len(variable_list1):]

  train_op1 = optimizer1.apply_gradients(zip(grads1, variable_list1))
  train_op2 = optimizer2.apply_gradients(zip(grads2, variable_list2))

  train_op = tf.group(train_op1, train_op2)

  train_loss(loss)
  train_accuracy(labels, predictions)

In [23]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  labels = tf.one_hot(labels, 31)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [24]:
for epoch in range(MAX_EPOCH):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  iter_source = iter(dset_loaders["source_tr"])
  for i, _ in enumerate(dset_loaders["source_tr"]):
    inputs_source, labels_source = iter_source.next()
    inputs_source = inputs_source.permute(0, 2, 3, 1)
    train_step(inputs_source.numpy(), labels_source.numpy())

  if (epoch+1) % 10 == 0 :
    iter_source = iter(dset_loaders["test"])
    for i, _ in enumerate(dset_loaders["test"]):
      inputs_source, labels_source = iter_source.next()
      inputs_source = inputs_source.permute(0, 2, 3, 1)
      test_step(inputs_source.numpy(), labels_source.numpy())

    lr1 = optimizer1._decayed_lr(tf.float32)
    lr2 = optimizer2._decayed_lr(tf.float32)
    print(
      f'Epoch {epoch + 1}, '
      f'lr1: {lr1:.5f}, '
      f'lr2: {lr2:.5f}, '
      f'Loss: {train_loss.result():.2f}, '  
      f'Accuracy: {train_accuracy.result() * 100:.2f}, '  
      f'Test Loss: {test_loss.result():.2f}, '
      f'Test Accuracy: {test_accuracy.result() * 100:.2f}'
    )

Epoch 10, lr1: 0.00059, lr2: 0.00595, Loss: 0.72, Accuracy: 100.00, Test Loss: 3.50, Test Accuracy: 4.76
Epoch 20, lr1: 0.00044, lr2: 0.00439, Loss: 0.70, Accuracy: 100.00, Test Loss: 3.49, Test Accuracy: 5.01
Epoch 30, lr1: 0.00035, lr2: 0.00354, Loss: 0.69, Accuracy: 100.00, Test Loss: 3.40, Test Accuracy: 7.74
Epoch 40, lr1: 0.00030, lr2: 0.00299, Loss: 0.69, Accuracy: 100.00, Test Loss: 3.34, Test Accuracy: 11.75
Epoch 50, lr1: 0.00026, lr2: 0.00261, Loss: 0.69, Accuracy: 100.00, Test Loss: 3.14, Test Accuracy: 17.64
Epoch 60, lr1: 0.00023, lr2: 0.00232, Loss: 0.69, Accuracy: 100.00, Test Loss: 2.95, Test Accuracy: 26.16
Epoch 70, lr1: 0.00021, lr2: 0.00210, Loss: 0.68, Accuracy: 100.00, Test Loss: 2.80, Test Accuracy: 35.07
Epoch 80, lr1: 0.00019, lr2: 0.00192, Loss: 0.69, Accuracy: 100.00, Test Loss: 2.58, Test Accuracy: 42.07
Epoch 90, lr1: 0.00018, lr2: 0.00178, Loss: 0.68, Accuracy: 100.00, Test Loss: 2.40, Test Accuracy: 45.83
Epoch 100, lr1: 0.00017, lr2: 0.00166, Loss: 0.68

In [25]:
#51-52%